# 导入数据

In [2]:
!pip install accelerate==0.21.0
!pip install transformers==4.24.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, classification_report,precision_recall_fscore_support
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback, set_seed
from tqdm import tqdm

In [ ]:
# 查看GPU信息
! /opt/bin/nvidia-smi

Thu May 30 13:26:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
balanced_df_all_cleaned = pd.read_csv('/content/drive/MyDrive/balanced_df_all_cleaned.csv')
# balanced_df_all_cleaned = pd.read_csv('/content/drive/MyDrive/balanced_df_all_cleaned_tokenization.csv')

In [8]:
# replace NaN values with an empty string
balanced_df_all_cleaned['tweet_content'] = balanced_df_all_cleaned['tweet_content'].fillna('')


In [9]:
df2=balanced_df_all_cleaned.copy()
# replace NaN values with an empty string
df2['tweet_content'] = df2['tweet_content'].fillna('')

In [ ]:
# after data balance
grouped_label = df2.groupby(['label', ]).size().reset_index(name='count')

In [ ]:
grouped_label

,label,count
0,0,10325
1,1,10325


In [12]:
# choose 2000 rows to retrain the models
df_label_0 = df2[df2['label'] == 0].sample(n=1000, random_state=43)
df_label_1 = df2[df2['label'] == 1].sample(n=1000, random_state=43)

# merge two dataframes
df3 = pd.concat([df_label_0, df_label_1])


In [10]:
# all rows
df3=df2

In [11]:
df3.shape

(20650, 3)

In [ ]:
df3.head()

,ID,label,tweet_content
0,25640,0,害真挺吃颜想岁足表达减肥决心姐妹谈恋爱感觉全世界闺蜜介绍成功行淦近新情侣装评手评寻思俩关系挺...
1,28303,0,拥抱世界暖心动作生活中难免挫折坎坷安慌乱许拥抱充满量世界抗衡愿生山树栖心爱春赏花夏纳凉秋登山...
2,10546,0,雨天穿滑倒坐水里亲测艾特子赟家意见麻雀妈妈问麻雀扎头发麻雀说啾啾晚子赟帮辫头发感找年糊图张回...
3,26929,0,幸福做桌菜然家朋友吃完没帮忙刷碗老规矩昨晚想做早餐奶包发酵忘遛弯回馒头样子朋友吃午两点吃早新...
4,31080,0,日清明青草疫情中牺牲医护员公安干警基层干部线工作逝世胞沉痛悼念情病毒面前选择逆行选择死生选择...


In [12]:

# Define pretrained tokenizer and model

model_name = "distilroberta-base" # distilroberta-base
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.

In [13]:
##############################
# Data: Text Input and Label #
##############################
X = list(df3["tweet_content"])
y = list(df3["label"])


In [14]:
########################
# Create torch dataset #
########################
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [15]:
# 方法一：定义计算评价指标的函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [16]:

# Train and Test Set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=112)


In [17]:

# Tokenize 用没有tokenized的数据集
# X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
# X_val_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)


In [ ]:
# Tokenize 用tokenized的数据集
# X_train_tokenized=X_train
# X_val_tokenized=X_test

In [18]:

########################
# Run cross-validation #
########################
def run_cross_validation(model_name='DistilRoBerta',
                         X=X,
                         y=y,
                         splits=5,# 默认是5
                         epoch=8,# 默认是8
                         checkpoint=False):

    kfold = StratifiedShuffleSplit(n_splits=splits, test_size=0.1, random_state=1127)# 默认test_size=0.1
    # kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=1127)
    n_fold = 1

    print("Developing Model with Cross validation for: " + model_name)
    for train, test in tqdm(kfold.split(X, y)):

        print("Running for Fold: ",n_fold)
        train_index = list(train)
        test_index = list(test)

        X_train = [X[i] for i in train_index]
        y_train = [y[i] for i in train_index]
        X_val = [X[i] for i in test_index]
        y_val = [y[i] for i in test_index]

        # Tokenize
        X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
        X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

        # Create torch dataset
        train_dataset = Dataset(X_train_tokenized, y_train)
        val_dataset = Dataset(X_val_tokenized, y_val)

        # Fine Tune Transformer
        # Define Trainer
        args = TrainingArguments(
            output_dir="content/drive/MyDrive_binary/output_" + model_name + "/fold"+str(n_fold),
            evaluation_strategy="epoch",
            save_strategy="epoch",
            #eval_steps=500,
            #per_device_train_batch_size=1,
            #per_device_eval_batch_size=1,
            num_train_epochs=epoch, #1 was okay
            seed=1127,
            load_best_model_at_end=True,
        )

        trainer = Trainer(
        #model_init=model_init,
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
        )

        if n_fold <=4:# 默认是4
            checkpoint_temp=True # 当设置为 true 时，会在训练过程中生成临时检查点文件，以便在发生意外情况（如程序崩溃）时可以从最近的检查点恢复训练。
        else:
            checkpoint_temp=False

        trainer.train(resume_from_checkpoint=checkpoint)
        print("Complete for fold", n_fold)
        n_fold= n_fold + 1

In [19]:

# Train and Validate Set

# Create torch dataset
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_test)

def run_hold_out_split(model_name='DistilRoBerta',
                       epoch=8,# 默认是8
                       train_dataset=train_dataset,
                       eval_dataset=val_dataset,
                       checkpoint=False):

    print("Developing Model with Hold Out Splits for: " + model_name)
    # Fine Tune Transformer
    # Define Trainer
    args = TrainingArguments(
      output_dir="content/drive/MyDrive_binary/output_" + model_name + "/holdout",
      evaluation_strategy="epoch",
      save_strategy="epoch",
      #eval_steps=500,
      #per_device_train_batch_size=1,
      #per_device_eval_batch_size=1,
      num_train_epochs=epoch, #1 was okay
      seed=1127,
      load_best_model_at_end=True,
    )

    trainer = Trainer(
    # model_init=model_init,
    args=args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    trainer.train(resume_from_checkpoint=checkpoint)
    print("Complete for hold-out validate set")


In [20]:
# 第五次运行，test size=0.1, no tokenization数据集, all rows
run_cross_validation(model_name='DistilRoBerta',
                         X=X,
                         y=y,
                         splits=2,# 折数
                         epoch=8,# 训练两轮
                         checkpoint=False)# 不保存检查点

Developing Model with Cross validation for: DistilRoBerta


0it [00:00, ?it/s]

Running for Fold:  1


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18585
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18592
  Number of trainable parameters = 82119938


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.420500,0.466977,0.855206,0.846226,0.868345,0.857143
2,0.346400,0.338095,0.874092,0.957396,0.783156,0.861555
3,0.340000,0.318182,0.873123,0.862653,0.887706,0.875000
4,0.318200,0.387323,0.887167,0.918410,0.849952,0.882856
5,0.287200,0.329228,0.885714,0.909558,0.856728,0.882353
6,0.292800,0.328018,0.886683,0.949381,0.817038,0.878252


...

Complete for fold 1
Running for Fold:  2


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18585
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18592
  Number of trainable parameters = 82119938


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.363800,0.338542,0.870218,0.903056,0.829622,0.864783
2,0.341300,0.409069,0.875545,0.861940,0.894482,0.877910
3,0.301300,0.373097,0.878450,0.943311,0.805421,0.868930
4,0.291100,0.332902,0.882324,0.917548,0.840271,0.877211
5,0.280800,0.379357,0.893462,0.921244,0.860600,0.889890
6,0.248600,0.345591,0.891041,0.914784,0.862536,0.887892
7,0.243600,0.354922,0.890557,0.924290,0.850920,0.886089


***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-2324
Configuration saved in content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-2324/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-2324/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-4648
Configuration saved in content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-4648/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-4648/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2065
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-6972
Configuration saved in content/drive/MyDrive_binary/o

Complete for fold 2


In [ ]:
#####################
# Run Hold Out Test #
#####################

# Train and Test Set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=1127)

# Train and Validate Set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train, random_state=1127)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_test)

def run_hold_out_split(model_name='DistilBert',
                       epoch=8,
                       train_dataset=train_dataset,
                       eval_dataset=val_dataset,
                       checkpoint=False):

    print("Developing Model with Hold Out Splits for: " + model_name)
    # Fine Tune Transformer
    # Define Trainer
    args = TrainingArguments(
      output_dir="content/drive/MyDrive_binary/output_" + model_name + "/holdout",
      evaluation_strategy="epoch",
      save_strategy="epoch",
      #eval_steps=500,
      #per_device_train_batch_size=1,
      #per_device_eval_batch_size=1,
      num_train_epochs=epoch, #1 was okay
      seed=1127,
      load_best_model_at_end=True,
    )

    trainer = Trainer(
    # model_init=model_init,
    args=args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    )

    trainer.train(resume_from_checkpoint=checkpoint)
    print("Complete for hold-out validate set")


In [ ]:
#holdout
run_hold_out_split(model_name='DistilBert',
                       epoch=8,
                       train_dataset=train_dataset,
                       eval_dataset=val_dataset,
                       checkpoint=False)

In [33]:
# 第四次运行，test size=0.1, no tokenization数据集, 2000 rows
run_cross_validation(model_name='DistilRoBerta',
                         X=X,
                         y=y,
                         splits=2,# 折数
                         epoch=8,# 训练两轮
                         checkpoint=False)# 不保存检查点

Developing Model with Cross validation for: DistilRoBerta


0it [00:00, ?it/s]

Running for Fold:  1


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1800
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1800
  Number of trainable parameters = 82119938


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.760602,0.815000,0.943662,0.670000,0.783626
2,No log,0.545229,0.815000,0.853933,0.760000,0.804233
3,0.476500,0.657845,0.765000,0.730435,0.840000,0.781395
4,0.476500,0.588308,0.810000,0.803922,0.820000,0.811881
5,0.392500,0.640114,0.820000,0.820000,0.820000,0.820000


...

Complete for fold 1
Running for Fold:  2


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1800
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1800
  Number of trainable parameters = 82119938


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.443762,0.870000,0.911111,0.820000,0.863158
2,No log,0.434019,0.875000,0.864078,0.890000,0.876847
3,0.407200,0.347511,0.880000,0.880000,0.880000,0.880000
4,0.407200,0.378908,0.900000,0.925532,0.870000,0.896907
5,0.340800,0.342449,0.895000,0.915789,0.870000,0.892308
6,0.340800,0.449482,0.890000,0.890000,0.890000,0.890000
7,0.293700,0.442674,0.885000,0.873786,0.900000,0.886700
8,0.293700,0.427977,0.895000,0.898990,0.890000,0.894472


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-225
Configuration saved in content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-225/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-225/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-450
Configuration saved in content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-450/config.json
Model weights saved in content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-450/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to content/drive/MyDrive_binary/output_DistilRoBerta/fold2/checkpoint-675
Configuration saved in content/drive/MyDrive_binary/output_Dist

Complete for fold 2


In [34]:
# 第四次运行，test size=0.1, no tokenization数据集, 2000 rows
run_hold_out_split(model_name='DistilRoBerta',
                       epoch=8,
                       train_dataset=train_dataset,
                       eval_dataset=val_dataset,
                       checkpoint=True)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Developing Model with Hold Out Splits for: DistilRoBerta


ValueError: No valid checkpoint found in output directory (content/drive/MyDrive_binary/output_DistilRoBerta/holdout)